In [1]:
from kronecker import KroneckerSolver
import kernels as kern
from likelihoods import PoissonLike, GaussianLike
import data_utils as sim
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from IPython.display import display
init_notebook_mode(connected=True)
import numpy as np
from kernels import RBF
from grid_utils import fill_grid
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()
import pandas as pd

In [11]:
counts = pd.read_csv('data/zip_counts.csv')
counts = counts[(counts['LAT'] > 20) & (counts['LAT'] < 50) & (counts['LNG'] > -130) & (counts['LNG'] < -70) &(counts['population'] > 0)]
step = .25
to_bin = lambda x: np.floor(x / step) * step
counts["latbin"] = counts.LAT.map(to_bin)
counts["lonbin"] = counts.LNG.map(to_bin)
grouped = pd.DataFrame(counts.groupby(['latbin', 'lonbin', 'committee_name'], as_index = False)['count', 'population'].sum())
clinton = grouped[grouped['committee_name'] == 'HILLARY FOR AMERICA']
X_clinton = np.array(clinton[['latbin','lonbin']])
y_clinton = np.array(clinton['count'], dtype = np.float32)
iplot([go.Scatter3d(x = X_clinton[:,1], y = X_clinton[:,0], z=y_clinton, mode = 'markers', marker=dict(size = 2,))])

In [12]:
X_full, y_full, obs_idx, imag_idx = fill_grid(X_clinton, y_clinton)
y_tf = tfe.Variable(y_full[obs_idx], dtype = tf.float32)
mu = tf.ones([X_full.shape[0]], tf.float32)*np.mean(np.log(y_full[obs_idx]))

In [17]:
ks = KroneckerSolver(mu, RBF(variance=5.0, length_scale=5.),
                     PoissonLike(), X_full, y_tf, obs_idx=obs_idx, verbose = True)

In [18]:
ks.run(10)
iplot([go.Scatter3d(x = X_full[obs_idx,1], y = X_full[obs_idx,0], z=ks.f_pred.numpy()[obs_idx], mode = 'markers', marker=dict(size = 2))])

Iteration:  <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>
 psi:  tf.Tensor(-1.05639e+07, shape=(), dtype=float32)
step 0.001953125

Iteration:  tf.Tensor(1, shape=(), dtype=int32)
 psi:  tf.Tensor(-1.39402e+07, shape=(), dtype=float32)
step 0.03125

Iteration:  tf.Tensor(2, shape=(), dtype=int32)
 psi:  tf.Tensor(-2.00919e+07, shape=(), dtype=float32)
step 0.25

Iteration:  tf.Tensor(3, shape=(), dtype=int32)
 psi:  tf.Tensor(-2.23102e+07, shape=(), dtype=float32)
step 1.0

Iteration:  tf.Tensor(4, shape=(), dtype=int32)
 psi:  tf.Tensor(-2.25137e+07, shape=(), dtype=float32)
step 2.0

Iteration:  tf.Tensor(5, shape=(), dtype=int32)
 psi:  tf.Tensor(-2.2525e+07, shape=(), dtype=float32)
step 1.0

Iteration:  tf.Tensor(6, shape=(), dtype=int32)
 psi:  tf.Tensor(-2.25397e+07, shape=(), dtype=float32)
step 1.0

Iteration:  tf.Tensor(7, shape=(), dtype=int32)
 psi:  tf.Tensor(-2.25398e+07, shape=(), dtype=float32)
step 0.25

Iteration:  tf.Tensor(8, shape=(), dtype=int32)
 psi: 

In [6]:
var = ks.variance(20).numpy()

KeyboardInterrupt: 

In [ ]:
iplot([go.Scatter3d(x = X_full[obs_idx,1], y = X_full[obs_idx,0], z=var, mode = 'markers', marker=dict(size = 2))])